In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('./src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trainers import *

2023-02-16 02:09:25.101955: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/ihidalgo/miniconda3/envs/prueba/lib/python3.7/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/ihidalgo/miniconda3/envs/prueba/lib/python3.7/site-packages/lpips/weights/v0.1/vgg.pth
2.4.1


In [ ]:
dataset_config = {'EM': [None, 'train', None, None, None, 'test'],
                  'MitoTracker_small': [None, 'train', None, None, None, 'test'],
                  }

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0";

dataset_name = 'EM'

train_lr, train_hr, val_lr, val_hr, test_lr, test_hr = dataset_config[dataset_name]

dataset_root = '../datasets'
train_lr_path = os.path.join(dataset_root, dataset_name, train_lr) if train_lr is not None else None
train_hr_path = os.path.join(dataset_root, dataset_name, train_hr) if train_hr is not None else None
val_lr_path = os.path.join(dataset_root, dataset_name, val_lr) if val_lr is not None else None
val_hr_path = os.path.join(dataset_root, dataset_name, val_hr) if val_hr is not None else None
test_lr_path = os.path.join(dataset_root, dataset_name, test_lr) if test_lr is not None else None
test_hr_path = os.path.join(dataset_root, dataset_name, test_hr) if test_hr is not None else None

scale = 4

model_configuration = {'optim': {'early_stop':{'loss':'val_ssim_loss','mode':'max', 'patience':10},
                                 'adam':{'beta1':0.5,'beta2':0.9,'epsilon':1e-07},
                                 'adamax':{'beta1':0.5,'beta2':0.9,'epsilon':1e-07},
                                 'adamW':{'decay':0.004,'beta1':0.5,'beta2':0.9,'epsilon':1e-07},
                                 'sgd_momentum':0.9,
                                 'ReduceOnPlateau':{'monitor':'val_loss','factor':0.5,'patience':3},
                                 'MultiStepScheduler':{'lr_steps':[50000, 100000, 200000, 300000],
                                                       'lr_rate_decay':0.5}},
                       'rcan': {'num_filters':16,
                                'percp_coef': 1000},
                       'dfcan': {'n_ResGroup': 4, 'n_RCAB': 4},
                       'wdsr': {'num_res_blocks': 32},
                       'unet': {'init_channels': 16,
                                'depth': 4,
                                'upsample_method': 'SubpixelConv2D',
                                'maxpooling': False,
                                'percp_coef': 10},
                       'wgan': {'g_layers': 15,
                                'd_layers': 5,
                                'recloss': 100.0,
                                'lambda_gp':10},
                       'esrganplus': {'n_critic_steps':5},
                       'others': {'positional_encoding':False,
                                  'positional_encoding_channels':64}
                      }

test_metric_indexes = [69,  7, 36, 75, 74, 30, 12, 42, 87,  0]

optimizer = 'Adam'  #'Adam', 'Adamax', 'RMSprop', 'SGD'
discriminator_optimizer = 'Adam'  #'Adam', 'Adamax', 'RMSprop', 'SGD'
scheduler = 'OneCycle'  #'ReduceOnPlateau', 'OneCycle', 'CosineDecay', 'MultiStepScheduler'
discriminator_lr_scheduler = 'OneCycle'  #'ReduceOnPlateau', 'OneCycle', 'CosineDecay', 'MultiStepScheduler'

crappifier_method = 'downsampleonly'

model_name = 'wgan' # ['unet', 'rcan', 'dfcan', 'wdsr', 'wgan', 'esrganplus']
seed = 666
batch_size = 2
number_of_epochs = 5
lr = 0.0001
discriminator_lr = 0.0001
additional_folder = "prueba"

num_patches = 16
patch_size_x = 64
patch_size_y = 64
validation_split = 0.1
data_augmentation = ['rotation', 'horizontal_flip', 'vertical_flip']

model = train_configuration(
                 data_name=dataset_name, 
                 train_lr_path=train_lr_path, train_hr_path=train_hr_path, 
                 val_lr_path=val_lr_path, val_hr_path=val_hr_path,
                 test_lr_path=test_lr_path, test_hr_path=test_hr_path,
                 crappifier_method=crappifier_method, 
                 model_name=model_name, scale_factor=scale, 
                 number_of_epochs=number_of_epochs, batch_size=batch_size, 
                 learning_rate=lr, discriminator_learning_rate=discriminator_lr, 
                 optimizer_name=optimizer, lr_scheduler_name=scheduler, 
                 test_metric_indexes=test_metric_indexes, 
                 additional_folder=additional_folder, 
                 model_configuration=model_configuration, seed=seed,
                 num_patches=num_patches, patch_size_x=patch_size_x, patch_size_y=patch_size_y, 
                 validation_split=validation_split, data_augmentation=data_augmentation,
                 discriminator_optimizer=discriminator_optimizer, 
                 discriminator_lr_scheduler=discriminator_lr_scheduler,
                 verbose=0
                )
                 

Simple discriminator
Generators parameters: 1443476
Discriminators parameters: 1554433


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LR patch shape: torch.Size([64, 64])
HR patch shape: torch.Size([256, 256])
configure_model() - lr 

Shape: (2, 1, 64, 64) 
Type: <class 'torch.Tensor'> 
Numpy type: float32 
Min: 0.012499013915657997 
Max: 0.8153286576271057 
Mean: 0.48789796233177185

configure_model() - hr 

Shape: (2, 1, 256, 256) 
Type: <class 'torch.Tensor'> 
Numpy type: float32 
Min: 0.0 
Max: 0.917647123336792 
Mean: 0.4879027307033539



LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name          | Type            | Params
--------------------------------------------------
0 | generator     | GeneratorModule | 1.4 M 
1 | discriminator | Discriminator   | 1.6 M 
2 | mae           | L1Loss          | 0     
--------------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.992    Total estimated model params size (MB)


Generator will use Adam optimizer.
Discriminator will use Adam optimizer.
Generator will use OneCycle scheduler
Discriminator will use OneCycle scheduler


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 0it [00:00, ?it/s]

(100, 1024, 1024, 1)
predict_images() - lr 

Shape: (1, 1, 256, 256) 
Type: <class 'torch.Tensor'> 
Numpy type: float32 
Min: 0.045906294137239456 
Max: 0.7973402738571167 
Mean: 0.5205608606338501

predict_images() - hr 

Shape: (1, 1, 1024, 1024) 
Type: <class 'torch.Tensor'> 
Numpy type: float32 
Min: 0.0 
Max: 0.8901961445808411 
Mean: 0.5205669403076172

predict_images() - predictions 

Not same shapes
Predicted images have been saved in: ./results/Electron microscopy/esrganplus/prueba/scale4/scale4_epc5_btch2_lr0.001_optim-Adam_lrsched-OneCycle_seed667/predicted_images
predict_images() - self.Y_test 

Shape: (100, 1024, 1024, 1) 
Type: <class 'numpy.ndarray'> 
Numpy type: float32 
Min: 0.0 
Max: 0.003921568859368563 
Mean: 0.002335971686989069

predict_images() - self.predictions 

Shape: (100, 1024, 1024, 1) 
Type: <class 'numpy.ndarray'> 
Numpy type: float32 
Min: 0.0 
Max: 1.0 
Mean: 0.9888588190078735

True HR shape: (100, 1024, 1024, 1)
Predicted HR shape: (100, 1024, 1024, 

100%|█████████████████████████████████████████| 100/100 [02:18<00:00,  1.38s/it]


ssim: 0.0057723490549940575
psnr: 0.06896725841126825
mse: 0.9842453704448474
alex: 0.413942813873291
vgg: 0.5236973166465759
ilniqe: nan
fsim: nan
gmsd: nan
vsi: nan
haarpsi: nan
mdsi: nan
pieapp: nan
dists: nan
brisqe: nan
successfully sent the mail
